<a href="https://colab.research.google.com/github/Arthur-ca/MIE1517/blob/Jing-Guo/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
import random
import torchvision
from torchvision import datasets, models
from collections import defaultdict
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as F1
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torchvision.datasets import ImageFolder
import cv2 as cv
# import tensorflow as tf
import gc
from collections import Counter
import imgaug.augmenters as iaa
import time

In [ ]:
#First link it to google drive and check the subfolder of the data
from google.colab import drive
drive.mount('/content/gdrive')
drive_content = os.listdir('/content/gdrive/My Drive/ColabNotebooks/archive')
print(drive_content)

In [ ]:
classeses = []
for i in os.listdir('C:/Users/Admin/Desktop/MIE1517_Project/data'):
    if i!=5:
        classeses.append(i)
classeses

In [ ]:
features_or = []
labels_or = []
train_data_or = []
data_transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
def create_train2():
    for i, photo in enumerate(classeses):
        path = os.path.join('C:/Users/Admin/Desktop/MIE1517_Project/data', photo)
        label = i  # Use the index of the class as the label

        # Loop over to get every image in the current class
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img_pil = cv.imread(img_path)
            img_pil = Image.fromarray(cv.cvtColor(img_pil, cv.COLOR_BGR2RGB))
            transformed_image1 = data_transform1(img_pil)

            if img_pil is None:
                print("Error loading image:", img_pil)
                continue
            # Append the original image and its label to the list
            features_or.append(transformed_image1)
            labels_or.append(label)

            train_data_or.append([img_pil, label])
create_train2()
print(type(features_or[0]))
# Count the occurrences of each label
label_counts_or = Counter(labels_or)

# Print the label counts
for label, count in label_counts_or.items():
    print(f"Label {label}: {count} images")


In [ ]:

# Define the classes (folders) in your dataset
classes = os.listdir('C:/Users/Admin/Desktop/MIE1517_Project/data')

# Initialize empty lists for features (transformed images) and labels
features = []
labels = []

data_transform2 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomRotation(degrees=(-50, 50)),
    transforms.ToTensor()
])
data_transform3 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
data_transform5 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.GaussianBlur(kernel_size=(7, 13), sigma=(9, 11)),  # The argument here specifies the radius, not kernel size
    transforms.ToTensor()
])


def create_train():
    for i, photo in enumerate(classes):
        path = os.path.join('C:/Users/Admin/Desktop/MIE1517_Project/data', photo)
        label = i  # Use the index of the class as the label

        # Loop over to get every image in the current class
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img_pil = cv.imread(img_path)

            if img_pil is None:
                print("Error loading image:", img_path)
                continue
            # Append the original image and its label to the list
            # features.append(np.array(img_array))
            # labels.append(label)

            # Apply the defined transformations
            img_pil = Image.fromarray(cv.cvtColor(img_pil, cv.COLOR_BGR2RGB))
            transformed_image1 = data_transform2(img_pil)
            #print(f"1:{type(transformed_image1)}")
            #transformed_image2 = data_transform2(img_pil)
            transformed_image3 = data_transform3(img_pil)
            #print(f"3:{type(transformed_image3)}")
            features.append(np.array(transformed_image1))
            #features.append(np.array(transformed_image2))
            features.append(np.array(transformed_image3))
            labels.append(label)
            labels.append(label)
            #labels.append(label)
            # train_data.append([transformed_image1, label])
            # #train_data.append([transformed_image2, label])
            # train_data.append([transformed_image3, label])
create_train()

# Count the occurrences of each label
label_counts = Counter(labels)

# Print the label counts
for label, count in label_counts.items():
    print(f"Label {label}: {count} images")

In [ ]:
n = 0
for images in features_or:
    print(f"Image Data Type: {type(images)}")
    n=n+1
    if n>10:
      break

In [ ]:
n = 0
for image in features_or:
    if isinstance(image, np.ndarray):
        image = torch.from_numpy(image).permute(2, 0, 1)
for images in features_or:
    print(f"Image Data Type: {type(images)}")
    print(f"Image Data Type: {images.shape}")
    n=n+1
    if n>10:
      break

In [ ]:
def show_images(images, labels):
    num_images = len(images)
    rows = int(np.sqrt(num_images))
    cols = int(np.ceil(num_images / rows))

    fig, axes = plt.subplots(rows, cols, figsize=(5, 5))

    for i, (image, label) in enumerate(zip(images, labels)):
        ax = axes.flatten()[i]
        ax.imshow(np.transpose(image, (1, 2, 0)))  # Swap axes for NumPy array
        ax.set_title(f"Label: {label}")
        ax.axis("off")

    plt.show()

# Iterate over the DataLoader
for label in range(len(classes)):  # Assuming 'classes' contains the class names
    # Filter images for the current label
    label_indices = [i for i, l in enumerate(labels) if l == label]

    # Randomly choose 5 images from the filtered indices
    chosen_indices = np.random.choice(label_indices, size=min(8, len(label_indices)), replace=False)

    # Extract images and labels for the chosen indices
    chosen_images = [features[i] for i in chosen_indices]
    chosen_labels = [labels[i] for i in chosen_indices]

    # Show images for the current label
    show_images(chosen_images, chosen_labels)

In [ ]:

from torchvision import transforms
from collections import defaultdict
labels = labels_or
features = features_or
# Shuffle the labels and features in the same order
combined = list(zip(labels, features))
labels, features = zip(*combined)

# Create a dictionary to store images by label
label_to_images = defaultdict(list)

# Group images by label
for label, img in zip(labels, features):
    label_to_images[label].append(img)

# Set the ratios for train, validation, and test sets
train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2

train_data = []
validation_data = []
test_data = []

# Split data for each label
for label, images in label_to_images.items():
    random.shuffle(images)  # Shuffle the images for each label

    num_images = len(images)
    num_train = int(train_ratio * num_images)
    num_validation = int(validation_ratio * num_images)
    num_test = num_images - num_train - num_validation

    train_data.extend([(img, label) for img in images[:num_train]])
    validation_data.extend([(img, label) for img in images[num_train:num_train + num_validation]])
    test_data.extend([(img, label) for img in images[num_train + num_validation:]])

In [ ]:

sample_index = 0  # Change this to the index of the sample you want to inspect

for i in range(15):
    sample_data, sample_label = test_data[i]
    print("Sample data shape:", sample_data.shape)

print("Sample label:", sample_label)
print(len(test_data))
print(len(validation_data))
print(len(train_data))

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img, label = self.data[index]

        # Convert img to PyTorch tensor
        img_tensor = torch.tensor(img, dtype=torch.float32)

        return img_tensor, label

# Create datasets
train_dataset = CustomDataset(train_data)
validation_dataset = CustomDataset(validation_data)
test_dataset = CustomDataset(test_data)

# Create DataLoader instances
batch_size = 64  # You can adjust this based on your needs
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check the tensor size
for images, labels in train_loader:
    print(images.size())  # This will print the size of the tensor
    break


```
Old Model and train func
```

In [ ]:
class CNNLargeNet(nn.Module):
    def __init__(self, output1=32, output2=64, output3=128, output4=256):
        super(CNNLargeNet, self).__init__()
        self.name = "large"
        
        # Define the sequential layers
        self.features = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(3, output1, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm2d(output1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.3),
            nn.MaxPool2d(2, 2),
            
            # Convolutional Layer 2
            nn.Conv2d(output1, output2, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(output2),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.3),
            nn.MaxPool2d(2, 2),
            
            # Convolutional Layer 3
            nn.Conv2d(output2, output3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(output3),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.3),
            nn.MaxPool2d(2, 2),
            
            # Convolutional Layer 4
            nn.Conv2d(output3, output4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(output4),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.3),
            nn.MaxPool2d(2, 2)
        )
        
        # Flatten the tensor for the fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(output4 * 14 * 14, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            
            nn.Linear(64, 8)  # Assuming 8 output classes
        )

    def forward(self, x):
        x = self.features(x)  # Pass the input through the feature layers
        x = x.view(x.size(0), -1)  # Flatten the tensor for the classifier
        x = self.classifier(x)  # Pass the flattened tensor through the classifier
        return x

In [ ]:
def trainmodel(model, train_dataset, val_dataset, batch=32, learningRate=0.01, num_epochs=500):
    # Create loss function and optimizer.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
    # Load data
    train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=True)
    # Create accuracy lists
    iters, losses, train_accuracy, validation_accuracy = [], [], [], []
    n = 0  # Initialize the iteration counter
    # Train the data
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_data in train_loader:
            imgs, labels = batch_data
            # img_to_tensor = transforms.ToTensor()
            # imgs = img_to_tensor(image)
            if use_cuda and torch.cuda.is_available():
                imgs = imgs.cuda()
                labels = labels.cuda()

            out = model(imgs)  # Forward pass
            loss = criterion(out, labels)  # Compute the total loss
            loss.backward()  # Backward pass (compute parameter updates)
            optimizer.step()  # Make the updates for each parameter
            optimizer.zero_grad()  # A clean-up step for PyTorch
            n += 1
            # print("Input image data type:", imgs.dtype)
            # print("Model parameters data types:")
            # for param in model.parameters():
            #     print(param.dtype)
            # Save the current training information
            running_loss += loss.item()

        iters.append(n)
        losses.append(running_loss / len(train_loader))  # Compute average loss for the epoch
        train_accuracy.append(get_accuracy(model, train_dataset))  # Compute training accuracy

        # Compute validation accuracy for this epoch
        validation_accuracy.append(get_accuracy(model, val_dataset))  # Compute validation accuracy

        # Print progress for the epoch
        print("Epoch {}, Train Accuracy: {:.6f}%, Validation Accuracy: {:.6f}%".format(
            epoch, train_accuracy[epoch], validation_accuracy[epoch]))

        # Save model checkpoint
        model_checkpoint_path = 'C:/Users/Admin/Desktop/MIE1517_Project/output'  # You can change the directory as needed
        model_checkpoint_file = os.path.join(model_checkpoint_path, f'{model.name} =Batch_size_{batch}_model_epoch{epoch + 1}.pth')
        torch.save(model.state_dict(), model_checkpoint_file)
        print(f"Saved model checkpoint: {model_checkpoint_file}")

    print("Finished Training")
    return iters, losses, train_accuracy, validation_accuracy

```
End Old Model and train func
```

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNLargeNet2(nn.Module):
    def __init__( self, output1=32, output2=64, output3=128, output4=256):
        super(CNNLargeNet2, self).__init__()
        self.name = "CNN"
        
        # Define the sequential layers for feature extraction
        self.features = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Convolutional Layer 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 5
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        
        # Flatten the tensor for the fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, 8)
        )

    def forward(self, x):
        x = self.features(x)  # Pass the input through the feature layers
        x = x.view(x.size(0), -1)  # Flatten the tensor for the classifier
        x = self.classifier(x)  # Pass the flattened tensor through the classifier
        return x

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, learning_rate, epoch)
    return path

def evaluate(model, loader):
    """ Evaluate the network """
    correct = 0
    total = 0

    for inputs, labels in loader:
      if use_cuda and torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()

      outputs = model(inputs)
      predicted = outputs.max(1, keepdim=True)[1]
      total += inputs.shape[0]
      correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return correct / total


In [ ]:
def trainmodel2(model, train_dataset, val_dataset, batch=64, learningRate=0.03, num_epochs=300):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
    
    model_checkpoint_path = 'C:/Users/Admin/Desktop/MIE1517_Project/output'  # You can change the directory as needed

    train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False) 

    train_accuracy = np.zeros(num_epochs)
    train_losses = np.zeros(num_epochs)
    validation_accuracy = np.zeros(num_epochs)
    validation_losses = np.zeros(num_epochs)
    iters = []

    # Check CUDA availability
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        model = model.cuda()

    n = 0  # Initialize the iteration counter
    # Training
    print("Training Begin...\n")
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        
        for i, data in enumerate(train_loader, 0):
            # Get the inputs
            inputs, labels = data
            # Set up for gpu running
            if use_cuda and torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass, backward pass, and optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            n += 1
            # Calculate loss
            running_loss += loss.item()

        train_losses[epoch] = running_loss
        train_accuracy[epoch] = evaluate(model, train_loader)

        model.eval()  # Set the model to evaluation mode for accuracy computation
        valid_loss = 0.0
        # Running without gradients are computed and model weights update
        for inputs, labels in val_loader:
            if use_cuda and torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()

        validation_losses[epoch] = valid_loss
        validation_accuracy[epoch] = evaluate(model, val_loader)
        iters.append(n)
        
        # Print progress for the epoch
        print(("Epoch {}: Train Accuracy: {}, Train loss: {} |"+
               "Validation Accuracy: {}, Validation loss: {}").format(
                   epoch + 1,
                   train_accuracy[epoch],
                   train_losses[epoch],
                   validation_accuracy[epoch],
                   validation_losses[epoch]))

        # Save model checkpoint every 5 epochs
        # if (epoch + 1) % 5 == 0:  # +1 because epochs start from 0
            # Save the current model checkpoint to a file
        model_path = get_model_name(model.name, batch_size, learningRate, epoch)
        model_path = os.path.join(model_checkpoint_path, model_path)
        torch.save(model.state_dict(), model_path)

    print("Finshied Training")
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    # Write the train/test loss/err into CSV file for plotting later
    # Save model checkpoint
    epochs = np.arange(1, num_epochs + 1)
    np.savetxt("{}_train_acc.csv".format(model_path), train_accuracy)
    np.savetxt("{}_train_loss.csv".format(model_path), train_losses)
    np.savetxt("{}_val_acc.csv".format(model_path), validation_accuracy)
    np.savetxt("{}_val_loss.csv".format(model_path), validation_losses)

    plt.title("Training Loss Curve")
    plt.plot(iters, train_losses, label="Train")
    plt.xlabel("iters")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Accuracy Curve")
    plt.plot(iters, train_accuracy, label="Train")
    plt.xlabel("iters")
    plt.ylabel("Accuracy")
    plt.show()
    print("Finished Training")
    return iters, train_losses, train_accuracy, validation_accuracy

In [ ]:
def trainSmall(model, data, batch_size=32, learning_rate=0.01, num_epochs=260):
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_acc = []

    # training
    print("Training Start")
    n = 0  # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):
            #############################################
            # To Enable GPU Usage
            imgs = imgs.cuda()
            labels = labels.cuda()
            #############################################

            # Check if imgs is already a tensor
            if not torch.is_tensor(imgs):
                img_to_tensor = transforms.ToTensor()
                imgs = img_to_tensor(imgs)

                # Convert input to float
                imgs = imgs.float()

            # Convert the input tensor to the desired type
            imgs = imgs.type(torch.float32)

            out = model(imgs)  # forward pass
            loss = criterion(out, labels)  # compute the total loss
            loss.backward()  # backward pass (compute parameter updates)
            optimizer.step()  # make the updates for each parameter
            optimizer.zero_grad()  # a clean-up step for PyTorch
            n += 1

        train_accuracy = get_accuracy(model, data)
        train_acc.append(train_accuracy)
        print("Epoch:{}, Accuracy:{}".format(epoch, train_accuracy))
    return train_acc


In [ ]:
batch_size = 3
num_workers = 1
small_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

# Visualize some sample data
classes = ['neutrality', 'sadness', 'fear', 'surprise', 'contempt', 'happiness', 'anger', 'disgust']
dataiter = iter(small_loader)
images, labels = next(dataiter)
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(3):
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])

    # If the image has three channels
    if images[idx].shape[0] == 3:
        plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    else:  # If the image is grayscale
        plt.imshow(images[idx][0], cmap='gray')

    ax.set_title(classes[labels[idx]])

In [ ]:
use_cuda = True
if torch.cuda.is_available():
    model = CNNLargeNet2().cuda()
Y = trainmodel2(model, train_data, validation_data)